----
----

# 🔥 Uygulama: C++ / LibTorch ile YOLO Tarzı Nesne Tespiti (Object Detection)

---
---

## 1️⃣ Python tarafı – YOLOv8 modelini hazırlama

Burada amacım şu:

* “YOLOv8’i Python’da alıp, C++ tarafında LibTorch ile çalışabilecek bir forma getireceğim.”

Normalde YOLOv8 için en rahat yol:

* ONNX → TensorRT / OpenVINO / başka runtime.

* Ama biz bu projede C++ + LibTorch teması üzerinden gidiyoruz. O yüzden 

## iki seçenek var:

## Seçenek A – PyTorch checkpoint + kendi forward wrapper’ını yazmak (LibTorch ile uyumlu Tensor çıkışı)

Bu senaryoda:

* Python’da YOLOv8 modelini PyTorch modülü olarak alıyorum.

* Üzerine kendi basitleştirilmiş forward fonksiyonumu yazıyorum:

>Input: [1,3,H,W]

>Output: sadeleştirilmiş bir tensor (mesela [num_boxes, 6] → x1,y1,x2,y2,score,class).

* Bu “sade” modülü TorchScript (script/trace) ile export ediyorum → yolov8_simplified.pt

C++ tarafında:

* torch::jit::load("yolov8_simplified.pt")

* Output zaten parse edilmiş halde olduğu için C++’ta decoding çok kolay oluyor.

**Bu, öğrenme amacı için en mantıklı yol.**

## Seçenek B – YOLOv8’in ham çıktılarını C++’ta decode etmek

* Modeli script/trace ediyorum ama forward sadece ham prediction döndürüyor.

C++ tarafında:

>Output şekli: [batch, num_boxes, no] (örneğin no = 84 → 4 bbox + 1 obj + 80 class gibi).

* Burada NMS, class score hesaplama vs. hep C++ tarafında.

### Burada benim önerim :

**Seçenek A:**
* Python’da YOLOv8 için şu amaçla küçük bir wrapper modül yazalım:

>forward(image) → tensor [N, 6] döndürsün (x1, y1, x2, y2, score, class_idx).

* Sonra bunu TorchScript’e çevirip C++’ta direkt kullanalım.

Böylece:

* YOLO iç matematiğini Python’da halledelim,

* C++ tarafı tamamen “sadece inference + çizim” kalıcak hale getirelim,

* Focus: C++ inference + proje akışı.

---
---


# 2️⃣ C++ tarafında tek projede iki mod (görsel + webcam)

Tek proje olacak, ana fikir şu:

* Projeye bir main.cpp koyacağım.

Programı çalıştırırken mode seçeceğim:

* ./yolo_app image → tek görsel üzerinde detection

* ./yolo_app webcam → canlı görüntü üzerinde detection

Bunun için main.cpp’de kabaca şöyle bir iskelet olacak:

```cpp
int main(int argc, char** argv)
{
    // 1) TorchScript YOLOv8 modelini yükle (yolov8_ts.pt)
    // 2) Argümanlara göre mod seç:
    //    - argc >= 2 && std::string(argv[1]) == "image"  -> tek görsel
    //    - argc >= 2 && std::string(argv[1]) == "webcam" -> kamera
    // 3) Ortak preprocess + inference fonksiyonlarını kullan
}
```

### Ve iki temel fonksiyon olacak:

>run_on_image(module, "input.jpg", "output.jpg");

>run_on_webcam(module);

* Böylece tek proje, tek exe ama iki kullanım modu:
```bash
# Tek görsel
yolo_app.exe image

# Webcam
yolo_app.exe webcam
```



----
---

## Önce python ile başlayalım 

In [1]:
import torch
from ultralytics import YOLO

def main():
    # 1) Hangi modeli kullanacağım?
    # yolov8n.pt (nano), yolov8s.pt (small) vs. seçebilirsin.
    model_name = "yolov8n.pt"

    print(f"[INFO] YOLOv8 modeli yukleniyor: {model_name}")
    model = YOLO(model_name)  # Ultralytics, hazır egitimli modeli indirip yüklüyor

    # 2) TorchScript'e export
    # format="torchscript" -> .torchscript uzantılı dosya üretir
    # imgsz=640 -> girdi boyutu (C++ tarafında da 640x640 kullanacağız)
    # nms=True  -> NMS'i export edilen modele gömer (C++ tarafını basitleştirir)
    print("[INFO] TorchScript formatina export ediliyor...")
    ts_path = model.export(
        format="torchscript",
        imgsz=640,
        nms=True,
        batch=1
    )

    print(f"[OK] TorchScript modeli olusturuldu: {ts_path}")
    print("Bu dosyayi C++ projemde model olarak kullanacagim.")

if __name__ == "__main__":
    # Gerekli paketler:
    # pip install ultralytics torch torchvision
    main()


[INFO] YOLOv8 modeli yukleniyor: yolov8n.pt
[INFO] TorchScript formatina export ediliyor...
Ultralytics 8.3.203  Python-3.10.18 torch-2.9.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 300, 6) (6.2 MB)

TorchScript: starting export with torch 2.9.1+cpu...
TorchScript: export success  1.9s, saved as 'yolov8n.torchscript' (12.5 MB)

Export complete (4.8s)
Results saved to C:\Users\hdgn5\OneDrive\Masast\C ++\Blm - 11 ( LibTorch )\Uygulama - 8
Predict:         yolo predict task=detect model=yolov8n.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8n.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
[OK] TorchScript modeli olusturuldu: yolov8n.torchscript
Bu dosyayi C++ projemde model olarak kullanacagim.


bulunduğun klasöre şuna benzer bir dosya düşecek:

* yolov8n.torchscript


İstersen adını değiştirip:

* yolov8_ts.pt


yaparsın, C++ tarafında o isimle çağırırız.

### Özet:
**“Ben burada herhangi bir eğitim yapmıyorum. YOLOv8’in zaten COCO’da eğitilmiş hazır ağırlıklarını alıp, onu TorchScript’e çeviriyorum.”**

----

# main.cpp – YOLOv8 TorchScript + LibTorch + OpenCV (image + webcam)

```cpp
// ---------------------------------------------------------
// YOLOv8 TorchScript + LibTorch + OpenCV
//  - Tek görselde nesne tespiti
//  - Webcam ile gerçek zamanlı nesne tespiti
// ---------------------------------------------------------

#include <iostream>
#include <fstream>
#include <string>
#include <vector>

#include <torch/script.h>
#include <opencv2/opencv.hpp>

// COCO sınıf isimleri (80 sınıf)
const std::vector<std::string> COCO_CLASSES = {
    "person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
    "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
    "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
    "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball",
    "kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket",
    "bottle","wine glass","cup","fork","knife","spoon","bowl","banana","apple",
    "sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake","chair",
    "couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote",
    "keyboard","cell phone","microwave","oven","toaster","sink","refrigerator",
    "book","clock","vase","scissors","teddy bear","hair drier","toothbrush"
};

// ---------------------------------------------------------
// Preprocess: cv::Mat (BGR) -> torch::Tensor [1, 3, 640, 640]
// ---------------------------------------------------------
torch::Tensor preprocess_image(const cv::Mat& img_bgr, int input_size = 640)
{
    cv::Mat img;

    // BGR -> RGB
    cv::cvtColor(img_bgr, img, cv::COLOR_BGR2RGB);

    // Resize (YOLOv8 genelde kare giriş kullanıyor: 640x640)
    cv::resize(img, img, cv::Size(input_size, input_size));

    // uint8 -> float32, 0-1
    img.convertTo(img, CV_32F, 1.0f / 255.0f);

    // HWC -> CHW tensöre çevir
    auto tensor_image = torch::from_blob(
        img.data,
        {1, input_size, input_size, 3}, // [N,H,W,C]
        torch::TensorOptions().dtype(torch::kFloat32));

    // NHWC -> NCHW
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [1,3,640,640]

    // contiguous + clone: OpenCV belleğinden ayrıştır
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}

// ---------------------------------------------------------
// YOLOv8 forward çıktılarını tensöre çevirme
// (Ultralytics export formatına göre esnek davranıyoruz)
// ---------------------------------------------------------
torch::Tensor get_detections_tensor(const torch::jit::IValue& output)
{
    if (output.isTensor())
    {
        return output.toTensor();
    }
    else if (output.isTuple())
    {
        auto elements = output.toTuple()->elements();
        if (!elements.empty() && elements[0].isTensor())
        {
            return elements[0].toTensor();
        }
    }
    else if (output.isList())
    {
        auto list = output.toList();
        if (list.size() > 0 && list.get(0).isTensor())
        {
            return list.get(0).toTensor();
        }
    }

    throw std::runtime_error("Beklenmeyen YOLOv8 output formati (Tensor/Tuple/List degil).");
}

// ---------------------------------------------------------
// Tek görsel üzerinde YOLOv8 inference ve bbox çizimi
// ---------------------------------------------------------
void run_on_image(torch::jit::script::Module& module,
                  const std::string& image_path,
                  const std::string& save_path = "output.jpg",
                  float conf_threshold = 0.25f)
{
    cv::Mat img_bgr = cv::imread(image_path);
    if (img_bgr.empty())
    {
        std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
        return;
    }

    std::cout << "[INFO] Resim yuklendi: " << image_path
              << "  Boyut: " << img_bgr.cols << "x" << img_bgr.rows << "\n";

    torch::NoGradGuard no_grad;

    // Preprocess
    torch::Tensor input_tensor = preprocess_image(img_bgr, 640);

    // Forward
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input_tensor);

    torch::jit::IValue output_iv = module.forward(inputs);

    // Çıktıyı tensöre çevir
    torch::Tensor det = get_detections_tensor(output_iv);

    // Beklenen çıkış: [num_dets, 6] veya [1, num_dets, 6]
    if (det.dim() == 3 && det.size(0) == 1)
    {
        det = det.squeeze(0); // [num_dets, 6]
    }

    if (!(det.dim() == 2 && det.size(1) >= 6))
    {
        std::cerr << "[WARN] Beklenmeyen detection tensor shape: " << det.sizes() << "\n";
        return;
    }

    std::cout << "[INFO] Toplam tespit sayisi: " << det.size(0) << "\n";

    int img_w = img_bgr.cols;
    int img_h = img_bgr.rows;

    for (int i = 0; i < det.size(0); ++i)
    {
        auto row = det[i];

        float x1 = row[0].item<float>();
        float y1 = row[1].item<float>();
        float x2 = row[2].item<float>();
        float y2 = row[3].item<float>();
        float score = row[4].item<float>();
        int cls_id   = static_cast<int>(row[5].item<float>());

        if (score < conf_threshold)
            continue;

        // Koordinatları görüntü boyutuna göre clamp et
        x1 = std::max(0.0f, std::min(x1, static_cast<float>(img_w - 1)));
        y1 = std::max(0.0f, std::min(y1, static_cast<float>(img_h - 1)));
        x2 = std::max(0.0f, std::min(x2, static_cast<float>(img_w - 1)));
        y2 = std::max(0.0f, std::min(y2, static_cast<float>(img_h - 1)));

        cv::Rect rect(cv::Point(static_cast<int>(x1), static_cast<int>(y1)),
                      cv::Point(static_cast<int>(x2), static_cast<int>(y2)));

        std::string label = "id=" + std::to_string(cls_id);
        if (cls_id >= 0 && cls_id < static_cast<int>(COCO_CLASSES.size()))
        {
            label = COCO_CLASSES[cls_id] + " " + cv::format("%.2f", score);
        }

        // Kutu ve label çiz
        cv::rectangle(img_bgr, rect, cv::Scalar(0, 255, 0), 2);
        int baseLine = 0;
        cv::Size label_size = cv::getTextSize(label, cv::FONT_HERSHEY_SIMPLEX, 0.5, 1, &baseLine);
        cv::Rect bg_rect(
            cv::Point(rect.x, rect.y - label_size.height - baseLine),
            cv::Size(label_size.width, label_size.height + baseLine));

        cv::rectangle(img_bgr, bg_rect, cv::Scalar(0, 255, 0), cv::FILLED);
        cv::putText(img_bgr, label,
                    cv::Point(rect.x, rect.y - baseLine),
                    cv::FONT_HERSHEY_SIMPLEX,
                    0.5, cv::Scalar(0, 0, 0), 1);
    }

    cv::imshow("YOLOv8 - Image", img_bgr);
    cv::imwrite(save_path, img_bgr);
    std::cout << "[INFO] Sonuc kaydedildi: " << save_path << "\n";
    std::cout << "[INFO] Pencereyi kapatmak icin bir tusa bas.\n";
    cv::waitKey(0);
}

// ---------------------------------------------------------
// Webcam üzerinde YOLOv8 inference
// ---------------------------------------------------------
void run_on_webcam(torch::jit::script::Module& module,
                   int cam_index = 0,
                   float conf_threshold = 0.25f)
{
    cv::VideoCapture cap(cam_index);
    if (!cap.isOpened())
    {
        std::cerr << "[HATA] Kamera acilamadi! Index: " << cam_index << "\n";
        return;
    }

    std::cout << "[INFO] Webcam acildi. Cikis icin 'q' veya ESC.\n";

    torch::NoGradGuard no_grad;

    while (true)
    {
        cv::Mat frame_bgr;
        cap >> frame_bgr;
        if (frame_bgr.empty())
        {
            std::cerr << "[WARN] Bos frame alindi, devam ediyorum...\n";
            continue;
        }

        int img_w = frame_bgr.cols;
        int img_h = frame_bgr.rows;

        torch::Tensor input_tensor = preprocess_image(frame_bgr, 640);

        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(input_tensor);

        torch::jit::IValue output_iv = module.forward(inputs);
        torch::Tensor det = get_detections_tensor(output_iv);

        if (det.dim() == 3 && det.size(0) == 1)
            det = det.squeeze(0);

        if (!(det.dim() == 2 && det.size(1) >= 6))
        {
            // Beklenmedik shape -> bu frame'i atla
            // (log yazmak istemezsen bu kısmı silebilirsin)
            // std::cerr << "[WARN] Beklenmeyen detection shape: " << det.sizes() << "\n";
        }
        else
        {
            for (int i = 0; i < det.size(0); ++i)
            {
                auto row = det[i];

                float x1 = row[0].item<float>();
                float y1 = row[1].item<float>();
                float x2 = row[2].item<float>();
                float y2 = row[3].item<float>();
                float score = row[4].item<float>();
                int cls_id   = static_cast<int>(row[5].item<float>());

                if (score < conf_threshold)
                    continue;

                x1 = std::max(0.0f, std::min(x1, static_cast<float>(img_w - 1)));
                y1 = std::max(0.0f, std::min(y1, static_cast<float>(img_h - 1)));
                x2 = std::max(0.0f, std::min(x2, static_cast<float>(img_w - 1)));
                y2 = std::max(0.0f, std::min(y2, static_cast<float>(img_h - 1)));

                cv::Rect rect(cv::Point(static_cast<int>(x1), static_cast<int>(y1)),
                              cv::Point(static_cast<int>(x2), static_cast<int>(y2)));

                std::string label = "id=" + std::to_string(cls_id);
                if (cls_id >= 0 && cls_id < static_cast<int>(COCO_CLASSES.size()))
                {
                    label = COCO_CLASSES[cls_id] + " " + cv::format("%.2f", score);
                }

                cv::rectangle(frame_bgr, rect, cv::Scalar(0, 255, 0), 2);
                int baseLine = 0;
                cv::Size label_size = cv::getTextSize(label, cv::FONT_HERSHEY_SIMPLEX, 0.5, 1, &baseLine);
                cv::Rect bg_rect(
                    cv::Point(rect.x, rect.y - label_size.height - baseLine),
                    cv::Size(label_size.width, label_size.height + baseLine));

                cv::rectangle(frame_bgr, bg_rect, cv::Scalar(0, 255, 0), cv::FILLED);
                cv::putText(frame_bgr, label,
                            cv::Point(rect.x, rect.y - baseLine),
                            cv::FONT_HERSHEY_SIMPLEX,
                            0.5, cv::Scalar(0, 0, 0), 1);
            }
        }

        cv::imshow("YOLOv8 - Webcam", frame_bgr);
        char key = static_cast<char>(cv::waitKey(1));
        if (key == 'q' || key == 27)
        {
            std::cout << "[INFO] Kullanici cikis istedi.\n";
            break;
        }
    }

    cap.release();
    cv::destroyAllWindows();
}

// ---------------------------------------------------------
// main: tek exe, iki mod
//   - image  -> tek görsel
//   - webcam -> canlı akış
// ---------------------------------------------------------
int main(int argc, char** argv)
{
    try
    {
        std::cout << "[INFO] YOLOv8 C++ uygulamasi basladi.\n";

        const std::string model_path = "yolov8n.torchscript"; // kendi dosya adina gore duzenle

        // Model dosyasi kontrol
        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] Model dosyasi bulunamadi: " << model_path << "\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }

        std::cout << "[INFO] Model yukleniyor: " << model_path << "\n";
        torch::jit::script::Module module = torch::jit::load(model_path);
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";

        std::string mode = "image";
        if (argc >= 2)
        {
            mode = std::string(argv[1]);
        }

        if (mode == "image")
        {
            std::string image_path = "input.jpg";
            if (argc >= 3)
            {
                image_path = std::string(argv[2]);
            }
            std::cout << "[INFO] Mod: IMAGE  |  Dosya: " << image_path << "\n";
            run_on_image(module, image_path, "output.jpg", 0.25f);
        }
        else if (mode == "webcam")
        {
            std::cout << "[INFO] Mod: WEBCAM\n";
            run_on_webcam(module, 0, 0.25f);
        }
        else
        {
            std::cerr << "[WARN] Bilinmeyen mod: " << mode << "\n";
            std::cerr << "Kullanim:\n";
            std::cerr << "  " << argv[0] << " image [resim_yolu]\n";
            std::cerr << "  " << argv[0] << " webcam\n";
        }

        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error& e)
    {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception& e)
    {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}
```


## Nasıl çalıştıracaksın?

### CMakeLists.txt
* Önceki LibTorch + OpenCV projelerinle aynı, sadece:
```bash
add_executable(yolov8_app main.cpp)
target_link_libraries(yolov8_app "${TORCH_LIBRARIES}" ${OpenCV_LIBS})
```


### Build (Release):
```bash
cmake --build . --config Release
```


Çalıştırmadan önce, build/Release klasörüne şunları koy:

* yolov8n.torchscript (Python’da export ettiğin dosya)

* input.jpg (test edeceğin bir resim)

* DLL’ler (LibTorch + OpenCV)

Tek görsel modu:
```bash
yolov8_app.exe image input.jpg
```


Webcam modu:
```bash
yolov8_app.exe webcam
```


----

## Eğer bu aşamaya kadar geldiyseniz ve proje doğru sonuçlandı ise kalan uygulama yada derslerden devam edebilirsiniz.Fakat sizde benim gibi sürümden kaybettiyseniz aşağıdaki markdownları detaylı inceleyiniz.
---

### Hata kodu:
```bash
[INFO] YOLOv8 C++ uygulamasi basladi.
[INFO] Model yukleniyor: yolov8_ts.pt
[EXCEPTION - std]
Unknown builtin op: torchvision::nms.
Could not find any similar ops to torchvision::nms. This op may not exist or may not be currently supported in TorchScript.
:
  File "code/__torch__/ultralytics/engine/exporter.py", line 48
    _17 = torch.slice(nmsbox0, 0, 0, 9223372036854775807)
    _18 = torch.slice(_17, 1, 4, 9223372036854775807)
    _19 = ops.torchvision.nms(torch.cat([offbox, _18], -1), score0, 0.69999999999999996)
          ~~~~~~~~~~~~~~~~~~~ <--- HERE
    keep = torch.slice(_19, 0, 0, 300)
    _20 = annotate(List[Optional[Tensor]], [keep])

Enter'a basip cikabilirsin...

```


Bu hata aslında bize şunu söylüyor:

* “Python tarafında export ettiğim YOLOv8 TorchScript modeli, içinde torchvision::nms (C++ tarafında bulunmayan bir op) kullanıyor. LibTorch bu op’u tanımıyor, o yüzden C++’ta çalışmıyor.”

### Yani sorun modelin içinde NMS olması. Biz ne yapmıştık?

Python tarafında export ederken:
```python
model.export(format="torchscript", imgsz=640, nms=True, batch=1)
```


demiştik.
* Bu nms=True yüzünden Ultralytics, NMS için torchvision::nms op’unu modele gömdü.
* LibTorch C++’ta ise torchvision C++ extension (libtorchvision) kurulu değil → op yok → patlıyor.

Çözüm net:
* 👉 NMS’i modelin içinden çıkaracağız (nms=False)
* 👉 YOLO çıktısını C++ tarafında kendimiz NMS ile işleyeceğiz.


# 1️⃣ Python tarafında modeli yeniden export et (NMS’siz)

* Eski TorchScript dosyan kullanılamaz artık, yenisini üretmemiz gerekiyor.


In [2]:
import torch
from ultralytics import YOLO

def main():
    model_name = "yolov8n.pt"  # veya kullandığın başka bir v8 ağırlığı

    print(f"[INFO] YOLOv8 modeli yukleniyor: {model_name}")
    model = YOLO(model_name)

    print("[INFO] TorchScript formatina export ediliyor (NMS YOK, C++ tarafinda yapilacak)...")
    ts_path = model.export(
        format="torchscript",
        imgsz=640,
        nms=False,    # <<< BURASI ÇOK ÖNEMLİ: NMS KAPALI
        batch=1
    )

    print(f"[OK] TorchScript modeli olusturuldu: {ts_path}")
    print("Bu dosyayi C++ projemde model olarak kullanacagim.")

if __name__ == "__main__":
    main()


[INFO] YOLOv8 modeli yukleniyor: yolov8n.pt
[INFO] TorchScript formatina export ediliyor (NMS YOK, C++ tarafinda yapilacak)...
Ultralytics 8.3.203  Python-3.10.18 torch-2.9.1+cpu CPU (AMD Ryzen 7 5800H with Radeon Graphics)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TorchScript: starting export with torch 2.9.1+cpu...
TorchScript: export success  1.4s, saved as 'yolov8n.torchscript' (12.5 MB)

Export complete (1.7s)
Results saved to C:\Users\hdgn5\OneDrive\Masast\C ++\Blm - 11 ( LibTorch )\Uygulama - 8
Predict:         yolo predict task=detect model=yolov8n.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8n.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
[OK] TorchScript modeli olusturuldu: yolov8n.torchscript
Bu dosyayi C++ projemde model olarak kullanacagim.


----

# C++ tarafında en basit postprocess’e geçiyoruz:

* “Ben YOLO çıktısından sadece en yüksek skorlu tek kutuyu alıyorum, NMS yok.”

Bunu yapmak için:

* Modeli nms=False ile tekrar export etmiş olman lazım (az önce konuştuğumuz gibi).

C++’ta ham çıktıyı alıp:

* Her satır için skor hesaplayacağız,

* En yüksek skorluyu seçeceğiz,

* Sadece o kutuyu çizeceğiz.

````cpp
// ---------------------------------------------------------
// YOLOv8 TorchScript + LibTorch + OpenCV
//  - Tek görselde nesne tespiti (en iyi tek kutu)
//  - Webcam ile gerçek zamanlı nesne tespiti (en iyi tek kutu)
//  - NMS yok, sadece en yüksek skorlu box seçiliyor
// ---------------------------------------------------------

#include <iostream>
#include <fstream>
#include <string>
#include <vector>
#include <algorithm>

#include <torch/script.h>
#include <opencv2/opencv.hpp>

// COCO sınıf isimleri (80 sınıf)
const std::vector<std::string> COCO_CLASSES = {
    "person","bicycle","car","motorcycle","airplane","bus","train","truck","boat",
    "traffic light","fire hydrant","stop sign","parking meter","bench","bird","cat",
    "dog","horse","sheep","cow","elephant","bear","zebra","giraffe","backpack",
    "umbrella","handbag","tie","suitcase","frisbee","skis","snowboard","sports ball",
    "kite","baseball bat","baseball glove","skateboard","surfboard","tennis racket",
    "bottle","wine glass","cup","fork","knife","spoon","bowl","banana","apple",
    "sandwich","orange","broccoli","carrot","hot dog","pizza","donut","cake","chair",
    "couch","potted plant","bed","dining table","toilet","tv","laptop","mouse","remote",
    "keyboard","cell phone","microwave","oven","toaster","sink","refrigerator",
    "book","clock","vase","scissors","teddy bear","hair drier","toothbrush"
};

// ---------------------------------------------------------
// Preprocess: cv::Mat (BGR) -> torch::Tensor [1, 3, input_size, input_size]
// ---------------------------------------------------------
torch::Tensor preprocess_image(const cv::Mat& img_bgr, int input_size = 640)
{
    cv::Mat img;

    // BGR -> RGB
    cv::cvtColor(img_bgr, img, cv::COLOR_BGR2RGB);

    // Resize (YOLOv8 genelde kare giriş: 640x640)
    cv::resize(img, img, cv::Size(input_size, input_size));

    // uint8 -> float32, 0-1
    img.convertTo(img, CV_32F, 1.0f / 255.0f);

    // HWC -> NHWC tensöre çevir
    auto tensor_image = torch::from_blob(
        img.data,
        {1, input_size, input_size, 3}, // [N,H,W,C]
        torch::TensorOptions().dtype(torch::kFloat32));

    // NHWC -> NCHW
    tensor_image = tensor_image.permute({0, 3, 1, 2}); // [1,3,640,640]

    // contiguous + clone: OpenCV belleğinden ayrıştır
    tensor_image = tensor_image.contiguous().clone();

    return tensor_image;
}

// ---------------------------------------------------------
// YOLOv8 forward çıktılarını tensöre çevirme
// (TorchScript output: Tensor / Tuple / List olabilir)
// ---------------------------------------------------------
torch::Tensor get_detections_tensor(const torch::jit::IValue& output)
{
    if (output.isTensor())
    {
        return output.toTensor();
    }
    else if (output.isTuple())
    {
        auto elements = output.toTuple()->elements();
        if (!elements.empty() && elements[0].isTensor())
        {
            return elements[0].toTensor();
        }
    }
    else if (output.isList())
    {
        auto list = output.toList();
        if (list.size() > 0 && list.get(0).isTensor())
        {
            return list.get(0).toTensor();
        }
    }

    throw std::runtime_error("Beklenmeyen YOLOv8 output formati (Tensor/Tuple/List degil).");
}

// ---------------------------------------------------------
// En iyi tek box için struct
// ---------------------------------------------------------
struct BestDet {
    float x1, y1, x2, y2;
    float score;
    int cls_id;
    bool valid;
};

// ---------------------------------------------------------
// YOLOv8 ham çıktısından (nms=False) en iyi kutuyu seç
// Beklenen format (satır bazında kabaca):
//   [x, y, w, h, obj_conf, cls0_conf, cls1_conf, ...]
// ---------------------------------------------------------
BestDet get_best_box_from_yolo(const torch::Tensor& det,
                               int img_w, int img_h,
                               float conf_threshold)
{
    BestDet best;
    best.valid = false;
    best.score = 0.0f;
    best.cls_id = -1;

    if (det.dim() != 2 || det.size(0) == 0)
        return best;

    const int num_boxes = det.size(0);
    const int num_attrs = det.size(1);

    if (num_attrs < 6) {
        std::cerr << "[WARN] YOLO output attr sayisi cok dusuk: " << num_attrs << "\n";
        return best;
    }

    for (int i = 0; i < num_boxes; ++i)
    {
        auto row = det[i];

        float cx = row[0].item<float>();
        float cy = row[1].item<float>();
        float w  = row[2].item<float>();
        float h  = row[3].item<float>();

        float obj_conf = row[4].item<float>();

        // Sınıf skorları 5. indexten itibaren
        int best_cls = -1;
        float best_cls_score = 0.0f;

        for (int j = 5; j < num_attrs; ++j)
        {
            float cls_score = row[j].item<float>();
            if (cls_score > best_cls_score) {
                best_cls_score = cls_score;
                best_cls = j - 5; // 0–79 arası COCO sınıf indexi
            }
        }

        // Toplam skor = obj_conf * class_conf
        float total_score = obj_conf * best_cls_score;

        if (total_score < conf_threshold)
            continue;

        if (!best.valid || total_score > best.score)
        {
            // cx,cy,w,h -> x1,y1,x2,y2
            float x1 = cx - w / 2.0f;
            float y1 = cy - h / 2.0f;
            float x2 = cx + w / 2.0f;
            float y2 = cy + h / 2.0f;

            // Görüntü sınırlarına kırp
            x1 = std::max(0.0f, std::min(x1, static_cast<float>(img_w - 1)));
            y1 = std::max(0.0f, std::min(y1, static_cast<float>(img_h - 1)));
            x2 = std::max(0.0f, std::min(x2, static_cast<float>(img_w - 1)));
            y2 = std::max(0.0f, std::min(y2, static_cast<float>(img_h - 1)));

            best.x1 = x1;
            best.y1 = y1;
            best.x2 = x2;
            best.y2 = y2;
            best.score = total_score;
            best.cls_id = best_cls;
            best.valid = true;
        }
    }

    return best;
}

// ---------------------------------------------------------
// Tek görsel üzerinde YOLOv8 inference (en iyi tek box)
// ---------------------------------------------------------
void run_on_image(torch::jit::script::Module& module,
                  const std::string& image_path,
                  const std::string& save_path = "output.jpg",
                  float conf_threshold = 0.25f)
{
    cv::Mat img_bgr = cv::imread(image_path);
    if (img_bgr.empty())
    {
        std::cerr << "[HATA] Resim yuklenemedi: " << image_path << "\n";
        return;
    }

    std::cout << "[INFO] Resim yuklendi: " << image_path
              << "  Boyut: " << img_bgr.cols << "x" << img_bgr.rows << "\n";

    // Gösterim için 640x640'lik kopya
    cv::Mat vis;
    cv::resize(img_bgr, vis, cv::Size(640, 640));

    torch::NoGradGuard no_grad;

    // Preprocess
    torch::Tensor input_tensor = preprocess_image(img_bgr, 640);

    // Forward
    std::vector<torch::jit::IValue> inputs;
    inputs.push_back(input_tensor);

    torch::jit::IValue output_iv = module.forward(inputs);

    // Çıktıyı tensöre çevir
    torch::Tensor det = get_detections_tensor(output_iv);

    // Beklenen çıkış çoğunlukla [1, num_boxes, num_attrs]
    if (det.dim() == 3 && det.size(0) == 1)
    {
        det = det.squeeze(0); // [num_boxes, num_attrs]
    }

    if (det.dim() != 2)
    {
        std::cerr << "[WARN] Beklenmeyen detection tensor shape: " << det.sizes() << "\n";
        return;
    }

    int img_w = 640;
    int img_h = 640;

    BestDet best = get_best_box_from_yolo(det, img_w, img_h, conf_threshold);

    if (!best.valid)
    {
        std::cout << "[INFO] Threshold'u gecen kutu bulunamadi.\n";
    }
    else
    {
        cv::Rect rect(
            cv::Point(static_cast<int>(best.x1), static_cast<int>(best.y1)),
            cv::Point(static_cast<int>(best.x2), static_cast<int>(best.y2)));

        std::string label = "id=" + std::to_string(best.cls_id);
        if (best.cls_id >= 0 && best.cls_id < static_cast<int>(COCO_CLASSES.size()))
        {
            label = COCO_CLASSES[best.cls_id] + " " + cv::format("%.2f", best.score);
        }

        cv::rectangle(vis, rect, cv::Scalar(0, 255, 0), 2);

        int baseLine = 0;
        cv::Size label_size = cv::getTextSize(label, cv::FONT_HERSHEY_SIMPLEX, 0.5, 1, &baseLine);
        cv::Rect bg_rect(
            cv::Point(rect.x, rect.y - label_size.height - baseLine),
            cv::Size(label_size.width, label_size.height + baseLine));

        cv::rectangle(vis, bg_rect, cv::Scalar(0, 255, 0), cv::FILLED);
        cv::putText(vis, label,
                    cv::Point(rect.x, rect.y - baseLine),
                    cv::FONT_HERSHEY_SIMPLEX,
                    0.5, cv::Scalar(0, 0, 0), 1);
    }

    cv::imshow("YOLOv8 - Image (Best Box Only)", vis);
    cv::imwrite(save_path, vis);
    std::cout << "[INFO] Sonuc kaydedildi: " << save_path << "\n";
    std::cout << "[INFO] Pencereyi kapatmak icin bir tusa bas.\n";
    cv::waitKey(0);
}

// ---------------------------------------------------------
// Webcam üzerinde YOLOv8 inference (en iyi tek box)
// ---------------------------------------------------------
void run_on_webcam(torch::jit::script::Module& module,
                   int cam_index = 0,
                   float conf_threshold = 0.25f)
{
    cv::VideoCapture cap(cam_index);
    if (!cap.isOpened())
    {
        std::cerr << "[HATA] Kamera acilamadi! Index: " << cam_index << "\n";
        return;
    }

    std::cout << "[INFO] Webcam acildi. Cikis icin 'q' veya ESC.\n";

    torch::NoGradGuard no_grad;

    while (true)
    {
        cv::Mat frame_bgr;
        cap >> frame_bgr;
        if (frame_bgr.empty())
        {
            std::cerr << "[WARN] Bos frame alindi, devam ediyorum...\n";
            continue;
        }

        // Gösterim için 640x640'lik bir kopya
        cv::Mat vis;
        cv::resize(frame_bgr, vis, cv::Size(640, 640));
        int img_w = 640;
        int img_h = 640;

        torch::Tensor input_tensor = preprocess_image(frame_bgr, 640);

        std::vector<torch::jit::IValue> inputs;
        inputs.push_back(input_tensor);

        torch::jit::IValue output_iv = module.forward(inputs);
        torch::Tensor det = get_detections_tensor(output_iv);

        if (det.dim() == 3 && det.size(0) == 1)
            det = det.squeeze(0); // [num_boxes, num_attrs]

        if (det.dim() == 2)
        {
            BestDet best = get_best_box_from_yolo(det, img_w, img_h, conf_threshold);

            if (best.valid)
            {
                cv::Rect rect(
                    cv::Point(static_cast<int>(best.x1), static_cast<int>(best.y1)),
                    cv::Point(static_cast<int>(best.x2), static_cast<int>(best.y2)));

                std::string label = "id=" + std::to_string(best.cls_id);
                if (best.cls_id >= 0 && best.cls_id < static_cast<int>(COCO_CLASSES.size()))
                {
                    label = COCO_CLASSES[best.cls_id] + " " + cv::format("%.2f", best.score);
                }

                cv::rectangle(vis, rect, cv::Scalar(0, 255, 0), 2);
                int baseLine = 0;
                cv::Size label_size = cv::getTextSize(label, cv::FONT_HERSHEY_SIMPLEX, 0.5, 1, &baseLine);
                cv::Rect bg_rect(
                    cv::Point(rect.x, rect.y - label_size.height - baseLine),
                    cv::Size(label_size.width, label_size.height + baseLine));

                cv::rectangle(vis, bg_rect, cv::Scalar(0, 255, 0), cv::FILLED);
                cv::putText(vis, label,
                            cv::Point(rect.x, rect.y - baseLine),
                            cv::FONT_HERSHEY_SIMPLEX,
                            0.5, cv::Scalar(0, 0, 0), 1);
            }
        }

        cv::imshow("YOLOv8 - Webcam (Best Box Only)", vis);
        char key = static_cast<char>(cv::waitKey(1));
        if (key == 'q' || key == 27)
        {
            std::cout << "[INFO] Kullanici cikis istedi.\n";
            break;
        }
    }

    cap.release();
    cv::destroyAllWindows();
}

// ---------------------------------------------------------
// main: tek exe, iki mod
//   - image  -> tek görsel
//   - webcam -> canlı akış
// ---------------------------------------------------------
int main(int argc, char** argv)
{
    try
    {
        std::cout << "[INFO] YOLOv8 C++ uygulamasi basladi.\n";

        // TorchScript model dosyasi adi
        const std::string model_path = "yolov8_ts.pt"; // Python'da ne isimle kaydettiysen onu yaz

        // Model dosyasi kontrol
        {
            std::ifstream f(model_path);
            if (!f.good())
            {
                std::cerr << "[HATA] Model dosyasi bulunamadi: " << model_path << "\n";
                std::cout << "Enter'a basip cikabilirsin...\n";
                std::cin.get();
                return -1;
            }
        }

        std::cout << "[INFO] Model yukleniyor: " << model_path << "\n";
        torch::jit::script::Module module = torch::jit::load(model_path);
        module.to(torch::kCPU);
        module.eval();
        std::cout << "[OK] Model yuklendi ve eval modunda.\n";

        std::string mode = "image";
        if (argc >= 2)
        {
            mode = std::string(argv[1]);
        }

        if (mode == "image")
        {
            std::string image_path = "input.jpg";
            if (argc >= 3)
            {
                image_path = std::string(argv[2]);
            }
            std::cout << "[INFO] Mod: IMAGE  |  Dosya: " << image_path << "\n";
            run_on_image(module, image_path, "output.jpg", 0.25f);
        }
        else if (mode == "webcam")
        {
            std::cout << "[INFO] Mod: WEBCAM\n";
            run_on_webcam(module, 0, 0.25f);
        }
        else
        {
            std::cerr << "[WARN] Bilinmeyen mod: " << mode << "\n";
            std::cerr << "Kullanim:\n";
            std::cerr << "  " << argv[0] << " image [resim_yolu]\n";
            std::cerr << "  " << argv[0] << " webcam\n";
        }

        std::cout << "[INFO] Program bitti. Enter'a basip cikabilirsin...\n";
        std::cin.get();
    }
    catch (const c10::Error& e)
    {
        std::cerr << "[EXCEPTION - c10] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }
    catch (const std::exception& e)
    {
        std::cerr << "[EXCEPTION - std] " << e.what() << "\n";
        std::cout << "Enter'a basip cikabilirsin...\n";
        std::cin.get();
        return -1;
    }

    return 0;
}


---
---

## Son build:

---
---

## 1. build klasörünü oluşturdum

Projeye girip:
```bash
cd "C:\Users\hdgn5\OneDrive\Masaüstü\C ++\Bölüm - 11 ( LibTorch )\Uygulama - 8"
mkdir build
cd build
```


Bu adımda ben aslında şunu yaptım:

* “Kaynak kodlar üst klasörde kalsın, build çıktıları build/ içinde toplansın.”

## 2. CMake ile proje dosyalarını ürettim
```bash
cmake ..
```


Burada CMake şunları yaptı:

* CMakeLists.txt dosyamı okudu.

* yolov8_app isminde bir executable projesi oluşturdu.

* LibTorch ve OpenCV yollarını kullanarak derleyiciye gerekli include/lib ayarlarını çıkarttı.

* Visual Studio solution ve proje dosyalarını build içine yazdı.

Benim için anlamı:

- “Derleyicinin, LibTorch ve OpenCV ile bu projeyi nasıl derleyeceğini tarif eden yapı oluştu.”

## 3. Release modda derledim
```bash
cmake --build . --config Release
```


Bu komutla:

* main.cpp derlendi.

* build/Release klasörüne yolov8_app.exe üretildi.

**CMake’te yazdığım add_custom_command sayesinde:**

* C:/libtorch/bin içindeki LibTorch DLL’leri,

* C:/opencv/build/x64/vc16/bin içindeki OpenCV DLL’leri
exe’nin yanına kopyalandı.

Sonuç klasör:
```bash
build\Release\
    ├─ yolov8_app.exe      # C++ YOLOv8 uygulamam
    ├─ *.dll               # LibTorch + OpenCV runtime kütüphaneleri
    └─ (diğer VS/CMake dosyaları)

## 4. TorchScript YOLOv8 modelimi buraya koydum

Python tarafında nms=False ile export ettiğim modeli (örneğin):
```bash
yolov8_ts.pt
```


dosyasını aldım, exe ile aynı klasöre attım:
```bash
build\Release\
    ├─ yolov8_app.exe
    ├─ yolov8_ts.pt        # TorchScript YOLOv8 modeli
    ├─ *.dll
```


Bunun anlamı:

* “C++ kodunda model_path = "yolov8_ts.pt" dediğimde, exe ile aynı klasörde bu dosyayı bulabilsin.”

## 5. Test resmi koydum

Inference için kullanacağım resmi, mesela input.jpg, aynı klasöre attım:
```bash
build\Release\
    ├─ yolov8_app.exe
    ├─ yolov8_ts.pt
    ├─ input.jpg           # Tespit yapılacak resim
    ├─ *.dll
```

## 6. Programı çalıştırdım

**Varsayılan image modu için:***
```bash
yolov8_app.exe
```


veya açık açık:
```bash
yolov8_app.exe image
```


**dediğimde şunu yapmış oldum:**

* “YOLOv8 C++ uygulamamı, tek görsel modu ile input.jpg üzerinde çalıştırıyorum.”

**Eğer farklı resim kullanacaksam:**
```bash
yolov8_app.exe image kedi.jpg
```


**Eğer webcam modunu denemek istiyorsam:**
```bash
yolov8_app.exe webcam